## Importing necessary packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn import linear_model, svm
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, EarlyStopping

Using TensorFlow backend.


In [ ]:
np.random.seed(123)

## Reading files

In [2]:
data = scipy.io.loadmat('../datasets/P+M+Islamic.mat')  #Data_DL_F.mat

X = data['X_all'] # Input (Reflectance Spectra)
Y = data['Y_all'] # Labels

Y = Y-1

## Splitting data to training, validation and test sets

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1, stratify=Y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1, stratify=y_train)

## Maybe I need to normalize the data
X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
X_test = X_test.astype('float32')
#X_train /= 4095
#X_val /= 4095
#X_test /= 4095

## Processing the labels (one hot vector)

In [4]:
nb_classes = np.max(Y)+1 
# Convert 1-dimensional class arrays to n-dimensional class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_val = np_utils.to_categorical(y_val, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

## Defining the model

In [5]:
# create model
model = Sequential()
model.add(Dense(256, input_dim=221, activation='sigmoid'))
model.add(Dense(128, activation='sigmoid'))
model.add(Dense(64))
model.add(Dense(32))#, activation='tanh'))
#model.add(Dropout(0.25))
model.add(Dense(16))
model.add(Dense(nb_classes, activation='softmax'))

# Avoid overfitting
early_checker = EarlyStopping(monitor='val_acc', patience=5, verbose=0, mode='auto')

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [6]:
nb_epoch = 100
# Fit the model
model.fit(X_train, Y_train, epochs=nb_epoch, batch_size=32, verbose=1, validation_data=(X_val, Y_val), callbacks=[early_checker])

Train on 832 samples, validate on 208 samples
Epoch 1/100
832/832 [==============================] - 1s 1ms/step - loss: 2.4467 - acc: 0.1683 - val_loss: 2.2282 - val_acc: 0.2308
Epoch 2/100
832/832 [==============================] - 0s 265us/step - loss: 1.9547 - acc: 0.3425 - val_loss: 1.5846 - val_acc: 0.5529
Epoch 3/100
832/832 [==============================] - 0s 241us/step - loss: 1.3169 - acc: 0.5721 - val_loss: 1.0338 - val_acc: 0.7019
Epoch 4/100
832/832 [==============================] - 0s 254us/step - loss: 0.8614 - acc: 0.7752 - val_loss: 0.6925 - val_acc: 0.8077
Epoch 5/100
832/832 [==============================] - 0s 238us/step - loss: 0.5439 - acc: 0.9014 - val_loss: 0.4106 - val_acc: 0.9038
Epoch 6/100
832/832 [==============================] - 0s 245us/step - loss: 0.3431 - acc: 0.9567 - val_loss: 0.2664 - val_acc: 0.9327
Epoch 7/100
832/832 [==============================] - 0s 233us/step - loss: 0.2278 - acc: 0.9700 - val_loss: 0.1735 - val_acc: 0.9231
Epoch 8/100

## evaluate the model

In [7]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 0.0354232545082
Test accuracy: 1.0


## calculate predictions

In [8]:
predictions = model.predict(X_test)

In [10]:
Y_pred = y_classes = predictions.argmax(axis=-1)#model.predict_classes(X_test)